# CNN

## 구조
- 입력 레이어
- 합성곱 계층 + 활성화 함수
- 풀링 레이어
- 완전 연결 계층

![](https://cdn.analyticsvidhya.com/wp-content/uploads/2024/09/75211cnn-schema1.webp)  
[출처](https://www.analyticsvidhya.com/blog/2021/08/beginners-guide-to-convolutional-neural-network-with-implementation-in-python/)

In [1]:
import numpy as np
import gzip
import os
import os
import gzip
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torchvision import transforms


In [16]:
from urllib.request import urlretrieve

# # MNIST를 다운받을 경로
url = 'https://ossci-datasets.s3.amazonaws.com/mnist/'
# # MNIST를 저장할 디렉토리 (colab 사용 시, 기본 디렉토리는 `/content`)
dataset_dir = os.path.join(os.getcwd(), 'data')
# 
# # MNIST 데이터셋의 파일명 (딕셔너리)
key_file = {
    'train_img':'train-images-idx3-ubyte.gz',
    'train_label':'train-labels-idx1-ubyte.gz',
    'test_img':'t10k-images-idx3-ubyte.gz',
    'test_label':'t10k-labels-idx1-ubyte.gz'
}

# # 해당 경로가 없을 시 디렉토리 새로 생성
os.makedirs(dataset_dir, exist_ok=True)

# 해당 경로에 존재하지 않는 파일을 모두 다운로드
for filename in key_file.values():
    if filename not in os.listdir(dataset_dir):
        urlretrieve(url + filename, os.path.join(dataset_dir, filename))
        print("Downloaded %s to %s" % (filename, dataset_dir))

In [2]:
class MNIST:
    def __init__(self):
        self.model = self
        self.file_paths = {
            'train_img': 'train-images-idx3-ubyte.gz',
            'train_label': 'train-labels-idx1-ubyte.gz',
            'test_img': 't10k-images-idx3-ubyte.gz',
            'test_label': 't10k-labels-idx1-ubyte.gz'
        }
        self.data_dir = 'data'  # 데이터 폴더 경로
        self.dataset = self.load_dataset()

    def load_dataset(self):
        """데이터셋을 로드하는 함수"""
        dataset = {}
        for key, value in self.file_paths.items():
            if 'img' in key:
                dataset[key] = self.load_images(value)
            else:
                dataset[key] = self.load_labels(value)
        return dataset
    
    def load_labels(self, filename):
        """레이블 파일을 로드하는 함수"""
        path = os.path.join(self.data_dir, filename)
        with gzip.open(path, 'rb') as f:
            labels = np.frombuffer(f.read(), dtype=np.uint8, offset=8)
        return labels
    
    
    def load_images(self, filename):
        """이미지 파일을 로드하는 함수"""
        path = os.path.join(self.data_dir, filename)
        with gzip.open(path, 'rb') as f:
            data = np.frombuffer(f.read(), dtype=np.uint8, offset=16)
            images = data.reshape(-1, 28, 28)  # (N, 28, 28) 형식으로 변환
        return images
        

In [8]:

# 데이터 로더 클래스
class MNISTDataset(data.Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img = self.images[idx].astype(np.float32) / 255.0  # 정규화
        img = np.expand_dims(img, axis=0)  # (28, 28) -> (1, 28, 28)
        label = self.labels[idx]

        if self.transform:
            img = self.transform(img)

        return torch.tensor(img, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

# CNN 모델 정의
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 학습 함수
def training(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    return running_loss / len(train_loader)

# 평가 함수
def evaluation(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# 전체 학습 루프
def training_loop(model, train_loader, test_loader, criterion, optimizer, device, epochs):
    for epoch in range(epochs):
        train_loss = training(model, train_loader, criterion, optimizer, device)
        test_accuracy = evaluation(model, test_loader, device)

        print(f"Epoch {epoch+1}: Loss = {train_loss:.4f}, Test Accuracy = {test_accuracy:.2f}%")


In [9]:
# 데이터 로드
mnist = MNIST()
train_dataset = MNISTDataset(mnist.dataset['train_img'], mnist.dataset['train_label'])
test_dataset = MNISTDataset(mnist.dataset['test_img'], mnist.dataset['test_label'])

print(mnist.dataset['train_img'].shape, mnist.dataset['train_label'].shape, mnist.dataset['test_img'].shape, mnist.dataset['test_label'].shape)

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


In [10]:
train_loader = data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = data.DataLoader(test_dataset, batch_size=64, shuffle=False)

In [12]:
# 모델, 손실 함수, 옵티마이저 정의
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 실행
training_loop(model, train_loader, test_loader, criterion, optimizer, device, epochs=5)

Epoch 1: Loss = 0.1777, Test Accuracy = 98.46%
Epoch 2: Loss = 0.0533, Test Accuracy = 98.84%
Epoch 3: Loss = 0.0355, Test Accuracy = 98.78%
Epoch 4: Loss = 0.0274, Test Accuracy = 99.05%
Epoch 5: Loss = 0.0203, Test Accuracy = 99.07%
